# United States Immigration Data Analysis
### Data Engineering Capstone Project

#### Project Summary
This project’s objective was to establish an ETL pipeline for I94 immigration, earth surface temperatures, airport codes, and city demographic datasets to create an analytics data warehouse for US immigration trends.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Library Imports

In [12]:
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
The scope of this project is to create an analytics data warehouse to track U.S. Immigration Data. 
This analytics solution will help answer questions about trends in immigration. 
For example: 
- "Does the average temperature of a city make a city more appealing for immigrants?" or 
- "What is the most popular month for immigrating to the United States?" 

The tools utilized for this project include **Apache Spark and Amazon S3**. Spark will be used to read and process the data and S3 will store the fact and dimension tables created with a Snowflake architecture. 

#### Describe and Gather Data 
For this project, I will being using three datasets: I94 Immirgration Data, World Temperature Data, and U.S. Cities Demographic Data. For descriptions and exploration of these datasets, please refer to the cells below. 




---

#### I94 Immigration Data
---
This SAS dataset includes data pertaining to the I-94 Vistor Arrivals Program. The National Travel and Tourism Office work with the U.S. Department of Homeland Security to release this data. It contains information about each person that enters the United States from another Country. 


For more information about this dataset please refer to the [National Travel and Tourism Office website](https://www.trade.gov/national-travel-and-tourism-office).


In [4]:
# Read in the data here
df_i94 = pd.read_csv("immigration_data_sample.csv")

In [5]:
df_i94.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


#### World Temperature Data
---
This dataset comes from Kaggle. Stored in a CVS file, this file contains data about Global Land Temperatures by city. 

For more information about this dataset please refer to this [Kaggle Dataset](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).


In [6]:
# Read in the data here
df_temp = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')

In [7]:
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


#### U.S. Cities Demographic Data
---
This dataset, also stored as a CSV file, comes from opendatasoft and utilizes Census Bureau data from 2015.

For more information about this dataset please refer to the [opendatasoft Dataset](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).


In [8]:
# Read in the data here
df_demog = pd.read_csv("city_demo_data/us-cities-demographics.csv")

In [9]:
df_demog.head()

,City;State;Median Age;Male Population;Female Population;Total Population;Number of Veterans;Foreign-born;Average Household Size;State Code;Race;Count
0,Silver Spring;Maryland;33.8;40601;41862;82463;...
1,Quincy;Massachusetts;41.0;44129;49500;93629;41...
2,Hoover;Alabama;38.5;38040;46799;84839;4819;822...
3,Rancho Cucamonga;California;34.5;88127;87105;1...
4,Newark;New Jersey;34.6;138040;143873;281913;58...


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

---
#### I94 Immigration Data
A few fields in this dataset have many null values. These fields have been excluded from the dataframe before they are written to the parquet file in S3. Other fields we excluded that did not fit to desired model. 


**Fields excluded from the dataset:**
 - i94mode
 - count
 - dtadfile
 - entdepa
 - entdepd
 - entdepu
 - matflag
 - biryear
 - occup

All fields are defined as strings except arrdate and depdate. 

Arrdate and depdate have been converted to datetime from SAS date type but are stored as strings.


Duplicate values have been removed.

---
#### I94 SAS Labels Descriptions
Data from the file has been manually cleaned using Excel and saved into four csv files, located in the i94_sas_label_data directory, containing **key/value** matches for:
 - [City(Port)](i94_sas_label_data/i94_cities.csv) 
 - [State](i94_sas_label_data/i94_states.csv) 
 - [Visa](i94_sas_label_data/i94_visas.csv) 
 - [County](i94_sas_label_data/i94_countries.csv)


All fields are defined as strings. 


Duplicate values have been removed. 

---
#### World Temperature Data
To clean and wrangle this dataset, the following steps have been taken:
1. Limit the dataset to only include data from the U.S.
2. Convert dt string into date format and extract the year
3. Limit the dataset to include only data from the latest year in the dataset (2013)
4. Convert AverageTemperature to a double value for aggregation
5. Calculate the average temperature and convert the temperature from Celsius to Fahrenheit


To see more detailed steps taken to clean this data set, please refer to *clean_temperature_data* in [elt.py](elt.py).


**Fields excluded from this dataset are as follows:**
 - AverageTemperatureUncertainty
 - Latitude
 - Longitude

All fields are defined as strings, expect for Average Temperature that has been converted to a double for aggregation and dt that has been converted in year with format 'YYYY'. 


Duplicate values have been removed. 

---
#### U.S. Cities Demographics Data
Some fields have been excluded from this dataset to prevent redundancy and to preserve the data model.


**Fields excluded from the dataset:**
 - Number of Veterans
 - Race
 - Count

All fields are defined as strings. 


Duplicate values have been removed. 

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
![Conceptual Data Model](Immigration_Data_Model.jpg)
For this data warehouse, I chose to implement a snowflake model containing one Fact and five Dimension tables. This model was chosen to provide a structured, partially normalized approach to the data. The snowflake schema allows for children to have one or more parent tables and that coincided well with the chosen data sets. Snowflake schemas also allow one to many relationships, which was crutial for relationships between dimension tables. My goal was to partially normalize the data to make querying more intuitive, while still providing the bulk of the data in the fact table to foster faster performance.   


**fact_immigration_i94**

This fact table contains data from the I94 Immigration Dataset. The primary key is cic_id. It was chosen as the primary key because it was already included with the dataset and is unique to each record.

This table has seven foreign keys that refer to the dimension tables. The foreign keys primarily act as a reference to lookup codes stored in the fact table to prevent data redundancy.

---
**dim_county**

This dimension table contains data from I94 SAS Label Descriptions file and was manually processed into a csv file. The primary key is **country_cd**. It was chosen to be able to lookup country codes from the immigration table and get their associated name.

---
**dim_visa**

This dimension table contains data from I94 SAS Label Descriptions file and was manually processed into a csv file. The primary key is **visa_cd**. It was chosen to be able to lookup visa type codes from the immigration table and get their associated description.

---
**dim_state**

This dimension table contains data from I94 SAS Label Descriptions file and was manually processed into a csv file. The primary key is **state_cd**. It was chosen to be able to lookup visa type codes from the immigration table and get their associated name.

---
**dim_city**

This dimension table contains data from I94 SAS Label Descriptions file and was manually processed into a csv file. The primary key is a composite of **city_cd** and **city_name**. It was chosen to be able to lookup city codes from the immigration table and get their associated name as well as serve as a point of reference for cities in dim_city_demographics.

---
**dim_city_demographics**

This dimension table contains data from I94 SAS Label Descriptions file and was manually processed into a csv file.The primary key is a composite of **city** and **state_cd**. State_cd is a foreign key from dim_state. This table makes my design a Snowflake schema because it has multiple parents (City and State) and allows for a many to one relationship with the dim_state table. This was chosen to be able to lookup city and state codes from the associated dimension tables.

---

#### 3.2 Mapping Out Data Pipelines
1. Create a Spark session to aid the ELT processes
2. Process I94 SAS Label Descriptions data
 - Read, load the data in dataframes, and finally write the data to parquet files stored in S3 for dim_country, dim_city, dim_state, and dim_visa tables
3. Process City Demographics Data
 - Read City Demographics and World Temperature data, clean World Temperature data, join dataframes, and write the data to a parquet file stored in S3 for dim_city_demographics table
4. Process Immigration Data
  - Read, load the data in dataframe, and finally write the data to a parquet file stored in S3 for fact_immigration_i94 table

---

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [2]:
%run elt.py

read countries data
countries data has been written
read cities data
cities data has been written
read states data
states data has been written
read visas data
visas data has been written
read demographics data
read temperature data
temperature data is being cleaned
temperature data has been cleaned
demographics data has been written
read immigration data
immigration data has been written
Starting quality checks
Empty check passed. Table: dim_country_table is not empty. There are 289 rows.
Unique check passed. Table: dim_country_table has 289 unique rows.
Quality checks complete
Starting quality checks
Empty check passed. Table: dim_city_table is not empty. There are 603 rows.
Unique check passed. Table: dim_city_table has 603 unique rows.
Quality checks complete
Starting quality checks
Empty check passed. Table: dim_state_table is not empty. There are 55 rows.
Unique check passed. Table: dim_state_table has 55 unique rows.
Quality checks complete
Starting quality checks
Empty check pa

#### 4.2 Data Quality Checks

Two quality checks were performed on the tables in this data model:
1. Empty check: Verify that there are no empty rows in the table
2. Unique check: Verify that every row is unique in the table

Please refer to [quality_checks.py](quality_checks.py)

#### 4.3 Data dictionary 
The data dictionary is located in this [file](data_dictionary.md), please navigate there to view it.

---

### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### Tools and Technology

 - AWS S3: Data can be stored securely, affordably, and can easily be transfer to BI and Analytics Applications
 - Apache Spark (Pyspark): Can be used to read and transform big data, which is needed for the Immigration dataset
 - EMR Big Data Cluster (optional): Can be used to speed up the data processing and loading


#### Data Updates
 - Immigration data should be updated every month because that is the cadence of the raw dataset as well.
 - City Demographics data can be updated annually because that's how frequently it is collected by the government.
     - World Temperature data can be updated annually as well, if the data is available.
 - Data from SAS Label Data (Country, Visa, State, City) data does not need to be updated.

#### Scenarios
 - The data was increased by 100x:
     - An AWS EMR Cluster could be implemented to process the influx of data. Spark alone cannot handle that volume of data.
 - The data populates a dashboard that must be updated on a daily basis by 7am every:
     - The elt process can be rewritten using Apache Airflow. Airflow provides an intuitive way to schedule DAG runs that would easily solve this.
 - The database needed to be accessed by 100+ people
     - In order for 100+ people to access this data warehouse, it would need to be moved to Amazon Redshift - or another similar platform. 

#### Sample Query

The parquet files were copied into AWS Athena for sample querying. The sample query and results are as follows:

Note: the sample size Immigration Data was used for this query, actual results may be larger

Query:

SELECT cic_id, i94_year, i94_month, i94_city_cd, city_name, i94_state_cd, arr_date, dep_date,  ins_num, male_pop, female_pop, total_pop, avg_temperature, median_age, avg_hh_size
FROM fact_immigration_i94 i INNER JOIN dim_city c ON i.i94_city_cd = c.city_cd
LEFT OUTER JOIN dim_city_demographics cd ON c.city_name = cd.city 
AND i.i94_state_cd = cd.state_cd

Results:



In [20]:
query_results = pd.read_csv("query_results.csv")

In [31]:
pd.set_option('display.max_rows', 50)
query_results.head(50)

,cic_id,i94_year,i94_month,i94_city_cd,city_name,i94_state_cd,arr_date,dep_date,ins_num,male_pop,female_pop,total_pop,avg_temperature,median_age,avg_hh_size
0,4753503.0,2016.0,4.0,HHW,Honolulu,HI,2016-04-25,2016-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1396100.0,2016.0,4.0,HHW,Honolulu,HI,2016-04-08,2016-04-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5951265.0,2016.0,4.0,AGA,Agana,GU,2016-04-03,2016-04-05,3663.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4340220.0,2016.0,4.0,MIA,Miami,NaN,2016-04-23,2016-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4800385.0,2016.0,4.0,ATL,Atlanta,MI,2016-04-25,2016-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1782990.0,2016.0,4.0,NYC,New York,NY,2016-04-10,2016-04-18,NaN,4081698.0,4468707.0,8550405.0,53.8950,36.0,2.68
6,155381.0,2016.0,4.0,ATL,Atlanta,CA,2016-04-01,2016-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5512209.0,2016.0,4.0,SFR,San Francisco,CA,2016-04-29,2016-05-07,NaN,439752.0,425064.0,864816.0,61.2206,38.3,2.37
8,87211.0,2016.0,4.0,CHI,Chicago,GA,2016-04-01,2016-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,494733.0,2016.0,4.0,SFR,San Francisco,CA,2016-04-03,2016-05-15,NaN,439752.0,425064.0,864816.0,61.2206,38.3,2.37
